In [ ]:
//Oscar Delgado and Colins LISP projects 
//Souce: https://github.com/jaratec/spl/tree/master/src/main/scala/net/jaratec/spl

In [ ]:
//Imports
//import $ivy.`org.scala-lang.modules::scala-parser-combinators:1.1.2`
import scala.util.parsing.combinator._

In [ ]:
/*

Check List:
Type:
Booleans T - Defines
Numbers T scala.Int
Characters T - scala.Char
Strings T = String
Symbols T
Pairs and Lists T
Vectors F -
Procedures F -

*/

In [ ]:
// TYPES

sealed trait Bool
case object True extends Bool
case object False extends Bool

sealed trait List[+A]
case object Empty extends List[Nothing]
case class Cons[A](x : A, xs : List[A]) extends List[A]

sealed trait Pair[+A,+B]
case class MkPair[A,B](fst : A, snd : B) extends Pair[A,B]

sealed trait Symbols
case object EmptyS extends Symbols 
case class Extend(k : String, v : Expr, env : Symbols) extends Symbols

sealed trait Vectors
case object EmptyV extends Vectors
case class VecAppend(p : Int, v : Expr, env : Vectors) extends Vectors



//sealed trait Procedures

sealed trait Expr
case class LispNum(x : scala.Int) extends Expr
case class LispBin(x : Bool) extends Expr
case class Ident(x : String) extends Expr
case class Plus(x : Expr, y : Expr) extends Expr
case class Minus(x : Expr, y : Expr) extends Expr
case class Mult(x : Expr, y : Expr) extends Expr
case class Div(x : Expr, y : Expr) extends Expr
//case class **(x : Expr, y : Expr) extends Expr
case class Neg(x : Expr) extends Expr
case class Eq(x : Expr, y : Expr) extends Expr
case class And(x : Expr, y : Expr) extends Expr
case class Or(x : Expr, y : Expr) extends Expr
case class IfThenElse(p : Expr, t : Expr, f : Expr) extends Expr
case class Let(id : String, x : Expr, y : Expr) extends Expr
case class define(id: String, x : Expr) extends Expr
case class Call(id : Expr, x : Expr) extends Expr
case object Error extends Expr

In [ ]:
//This is colins box
//Its where I play
//On bitches booties

In [42]:
/*def VectorSize(v : Dictionary) : Int = v match{
    case EmptyD => 0
    case Vectors(i,v,xs) => i
    case _ => -1
}

def append(vec : Dictionary, v : Expr) : Vectors = vec match{
    case EmptyD => Vectors(1,v,EmptyD)
    case Vectors(i,v2,xs) => Vectors(i + 1, v , Vectors(i,v2,xs))
    case _ => Vectors(1,v,EmptyD)
}*/

sealed trait Maybe[+A]
case object Nothing extends Maybe[Nothing]
case class Just[A](fromJust : A) extends Maybe[A]

def lookup(dict : Symbols, k : String) : Maybe[Expr] = dict match {
    case EmptyS        => Nothing
    case Extend(k1, v, xs) => k == k1 match {
        case true  => Just(v)
        case false => lookup(dict, k)
    }
}

defined trait Maybe
defined object Nothing
defined class Just
lookup: (dict: Symbols, k: String)Maybe[Expr]


In [47]:
def bin_Arith(t : Char, v1 : Expr, v2 : Expr) : Expr = (t,v1,v2) match{
    case ('+',LispNum(v1),LispNum(v2)) => LispNum(v1+v2) 
    case ('-',LispNum(v1),LispNum(v2)) => LispNum(v1-v2) 
    case ('*',LispNum(v1),LispNum(v2)) => LispNum(v1*v2) 
    case ('/',LispNum(v1),LispNum(v2)) => LispNum(v1/v2) 
    case _ => Error
}
//def bin_Bool[a](f : ((Bool, Bool) => Bool), v1 : Value, v2 : Value) : Value = (v1,v2) match{
  //  case (BinVal(v1),BinVal(v2)) => BinVal(f(v1,v2))
    //case _ => Error  
//}

def Eval(store : Symbols, expr : Expr) : Expr = expr match{
    case LispNum(n) => LispNum(n)
    case LispBin(n) => LispBin(n)
    case Ident(x) => lookup(store , x) match{
        case Just(v) => v
        case _ => Error
    }
    case Plus(x,y) => bin_Arith('+' , Eval(store,x),Eval(store,y))
    case Minus(x,y) => bin_Arith('-' , Eval(store,x),Eval(store,y))
    case Mult(x,y) => bin_Arith('*' , Eval(store,x),Eval(store,y))
    case Div(x,y) => bin_Arith('/' , Eval(store,x),Eval(store,y))
    case Eq(x,y) => Eval(store,x) == Eval(store,y) match{
        case true => LispBin(True)
        case false => LispBin(False)
    }
    case Let(x,y,z) => Eval(Extend(x,Eval(store,y),store),z)
}

<console>:134: warning: match may not be exhaustive.
It would fail on the following inputs: And(_, _), Call(_, _), Error, IfThenElse(_, _, _), Neg(_), Or(_, _), define(_, _)
       def Eval(store : Symbols, expr : Expr) : Expr = expr match{
                                                       ^
bin_Arith: (t: Char, v1: Expr, v2: Expr)Expr
Eval: (store: Symbols, expr: Expr)Expr


In [44]:
Eval(EmptyS,Let("x" , LispNum(5) , Plus(Ident("x"),LispNum(5))))
//Eval(EmptyD , Plus(LispNum(5),LispNum(5)))

LispNum(10)

In [45]:
Same concept case -(x,y) => LispNum(x - y) 

def parser(thing : String) : List[Expr] = thing match{
    thing.split(" ") 
}

<console>: 1